In [1]:
import keras
import xmltodict
import os
import typing
import re
import matplotlib.pyplot as plt
import pandas as pd
import glob
import os
import math
import numpy as np
import cv2
import json
from pathlib import Path

Using TensorFlow backend.


In [2]:
#import all libraries
from PIL import Image, ImageOps,ImageFilter
#import cv2
import time
import argparse
from scipy import ndimage
from scipy import misc
%matplotlib inline
import readline
readline.parse_and_bind("tab: complete")
from scipy import ndimage
from multiprocessing.pool import ThreadPool

from urllib.request import Request, urlopen, HTTPError
import ssl

In [45]:
import os

def Get_folder_name_labels(dir_path):

    df_logos_folder = pd.DataFrame(columns=['true_logo','folder_name','folder_img_count'])
    mn = 145
    folders = ([name for name in sorted(os.listdir(dir_litw), key=str.casefold)]) # get all directories 
    num_folders =1

    for company_name in folders:
        contents = os.listdir(os.path.join(dir_path,company_name)) # get list of contents
        if len(contents) <= mn: # if greater than the limit, print folder and number of contents
            continue
        row = pd.Series({'true_logo' :num_folders,
                         'folder_name' :company_name,
                        'folder_img_count': len(contents),
                        'folder_path': os.path.join(dir_path,company_name),})
        print(os.path.join(dir_path,company_name))
        print("length:",str(len(contents)))
        num_folders = num_folders+1
        df_logos_folder = df_logos_folder.append(row,ignore_index=True)   
        
    
    return df_logos_folder

In [46]:
dir_litw=Path("c:/Users/Sanyogita/Documents/Insight/projects/Trademark_RADAR/large_data/brandROIs")
df_logos_folder_label=Get_folder_name_labels(dir_litw)
print("total folder", str(df_logos_folder_label.shape))

c:\Users\Sanyogita\Documents\Insight\projects\Trademark_RADAR\large_data\brandROIs\aldi
length: 245
c:\Users\Sanyogita\Documents\Insight\projects\Trademark_RADAR\large_data\brandROIs\allianz
length: 148
c:\Users\Sanyogita\Documents\Insight\projects\Trademark_RADAR\large_data\brandROIs\allianz-symbol
length: 150
c:\Users\Sanyogita\Documents\Insight\projects\Trademark_RADAR\large_data\brandROIs\aral
length: 163
c:\Users\Sanyogita\Documents\Insight\projects\Trademark_RADAR\large_data\brandROIs\audi-symbol
length: 229
c:\Users\Sanyogita\Documents\Insight\projects\Trademark_RADAR\large_data\brandROIs\bionade
length: 253
c:\Users\Sanyogita\Documents\Insight\projects\Trademark_RADAR\large_data\brandROIs\bmw
length: 222
c:\Users\Sanyogita\Documents\Insight\projects\Trademark_RADAR\large_data\brandROIs\bosch-symbol
length: 164
c:\Users\Sanyogita\Documents\Insight\projects\Trademark_RADAR\large_data\brandROIs\burgerking-symbol
length: 216
c:\Users\Sanyogita\Documents\Insight\projects\Trademark_R

In [47]:
#Image processing for resizing images
def fix_aspect_ratio(img,expected_dimension):
    original_max_dim = max(float(img.size[0]),float(img.size[1]))
    original_min_dim = min(float(img.size[0]),float(img.size[1]))
    wpercent = (expected_dimension/float(original_max_dim))
    hsize = int((original_min_dim*float(wpercent)))
    new_im = img.resize((expected_dimension,hsize), Image.ANTIALIAS)
    return new_im

In [48]:
# Make square images
def make_square(img, min_size=120, fill_color=(0, 0, 0, 0)):
    x, y = img.size
    size = max(min_size, x, y)
    new_im = Image.new('RGBA', (size, size), fill_color)
    new_im.paste(img, (int((size - x) / 2), int((size - y) / 2)))
    return new_im

In [49]:
# Sharpen the edges
from pylab import *
def sharpen_filter(img):
    sharp_im = img.filter(ImageFilter.SHARPEN)
    return sharp_im

In [50]:

def verify_images_df(glob_path_complete, folder_name,dir_litw_resized, size_threshold,folder_label,expected_dimension):
    try:
        df_logos_pickle = pd.DataFrame(columns=['true_logo','logo_img_name','original_image_name','logo_img_path','company_brand','brand_count','X_size','Y_size'])
         # in each folder, find image file and resize-scale them without distortion
        index_image= 1;

        glob_path_complete_path =Path(glob_path_complete)
    
        mod_folder_name = 'Mod-' + str(folder_name)
        mod_full_dir = os.path.join(dir_litw_resized,mod_folder_name) 
        brand_count_row=1
        for filename_logo in glob_path_complete_path.glob('**/*.jpg'):
            img = Image.open(filename_logo) # open the image file
            img.verify() # verify that it is, in fact an image
    except (IOError, SyntaxError) as e:
        print('Bad file:', filename_logo) # print out the names of corrupt files

In [75]:
# Load dataset of logos in a dataframe
def resize_images_df(glob_path_complete, folder_name,dir_litw_resized, size_threshold,folder_label,expected_dimension):
    df_logos_pickle = pd.DataFrame(columns=['true_logo','logo_img_name','original_image_name','logo_img_path','company_brand','brand_count','X_size','Y_size'])
     # in each folder, find image file and resize-scale them without distortion
    index_image= 1;
    #print(glob_path_complete)  
    glob_path_complete_path =Path(glob_path_complete)
    
    mod_folder_name = 'Mod-' + str(folder_name)
    mod_full_dir = os.path.join(dir_litw_resized,mod_folder_name) 
    brand_count_row=1
    for filename_logo in glob_path_complete_path.glob('**/*.jpg'):
         with open(filename_logo) as imagefile:
            image_original = Image.open(filename_logo)
            get_file_data_shape_x, get_file_data_shape_y = image_original.size
            max_size_dimension = max(float(get_file_data_shape_x),float(get_file_data_shape_y))
            #if max_size_dimension < size_threshold:
                #continue

            img_mod_name = folder_name +'-AB-'+ str(index_image) +'.png'
            
            os.makedirs(mod_full_dir, exist_ok=True)
            path_to_image = os.path.join(mod_full_dir,img_mod_name)
          
            
            company_brand_name = folder_name
            
           

            image_aspect=fix_aspect_ratio(image_original,expected_dimension)
            
            img_new = make_square(image_aspect)
            
            sharpen_image=sharpen_filter(img_new)
            im_med = ndimage.median_filter(sharpen_image, 3)
            
            switcher_output = folder_label
            
            row = pd.Series({'true_logo' :switcher_output,
                            'logo_img_name' :img_mod_name,
                             'original_image_path' :image_original,
                             'logo_img_path' :path_to_image,
                             'company_brand' :folder_name,
                             'brand_count' :brand_count_row,
                             'X_size' :get_file_data_shape_x,
                             'Y_size' :get_file_data_shape_y,})
            df_logos_pickle = df_logos_pickle.append(row,ignore_index=True)            
            brand_count_row=brand_count_row+1
            file_name=os.path.join(mod_full_dir,img_mod_name)
            misc.imsave(file_name, im_med)
            index_image =index_image+1
    print(brand_count_row)        
    return df_logos_pickle   
           
        

In [76]:
# Prepare the logos dataset datastructure
# Set threshold for the size of the logo images to avoid heavy distortion due to resize
def get_all_images(df_folder_label):

    dir_litw_resized=Path("c:/Users/Sanyogita/Documents/Insight/projects/Trademark_RADAR/large_data/resized")
    df_list=[]
    
    for index_df, record in df_folder_label.iterrows():
        expected_dimension=120
        folder_path_global=record['folder_path']
        size_threshold=16
        #verify_images_df(folder_path_global, record['folder_name'],dir_litw_resized, size_threshold,record['true_logo'],expected_dimension)
        df_list.append(resize_images_df(folder_path_global, record['folder_name'],dir_litw_resized, size_threshold,record['true_logo'],expected_dimension))
        tp_df = pd.concat(df_list, ignore_index=True)      
        
    return tp_df

In [78]:
tp_df_image_resized=get_all_images(df_logos_folder_label)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


246
149
151
164
230
254
223
165
217
155
243
206
242
235
241
225
181
221
268
225
244
257
237
228
158
236
226
168
198
163
224
212
240
176
220
223
147
250
191
230
201
215
221
166


In [84]:
import pickle
tp_df_image_resized.to_pickle('image_logos_folder_path.pkl')